

__Set your own route planner for a given OSM map and GTFS file and query server on localhost for a detailed routes__


### walkthrough

from: http://docs.opentripplanner.org/en/latest/Basic-Tutorial/

1. Install Java SDK
2. Download [latest](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/otp-1.4.0-shaded.jar) .jar from [Maven](https://repo1.maven.org/maven2/org/opentripplanner/otp/1.4.0/)
3. Download gtfs.zip of a given city into a `folder` from `transit.land`
4. Download osm.pgf of your area from [here](https://www.interline.io/osm/extracts/) (need to login to get API token)
5. call from terminal `java -Xmx6G -jar otp-1.4.0-shaded.jar --build data_path --inMemory` changing `6G` as memory allocated (6GB) and `data_path` to your path with osm and gtfs files
6. if succesful you shall be able to open a web search in your browser: `http://localhost:8080/`
7. and query it via API, for instance: 'http://localhost:8080/otp/routers/default/plan?fromPlace=52.25177%2C20.92758&toPlace=52.20550%2C21.00517&time=1%3A02pm&date=04-09-2020&mode=TRANSIT%2CWALK&maxWalkDistance=5000&arriveBy=false'


In [32]:
import pandas as pd
import requests

In [33]:
from main import make_query, parse_OTP_response
from utils import plot

set up

In [38]:
PATH = 'georequests.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

input

In [37]:
df = pd.read_csv(PATH, index_col=[0])  # load the csv
df.treq = pd.to_datetime(df.treq)
df.sample(5)[['origin_x','origin_y','destination_x','destination_y','treq']]

,origin_x,origin_y,destination_x,destination_y,treq
54,4.896151,52.379797,4.853960,52.382400,2020-10-29 14:32:07
563,4.870579,52.364917,4.825666,52.381687,2020-10-29 15:09:03
332,4.838084,52.346315,4.873289,52.361182,2020-10-29 15:26:59
490,4.890398,52.365332,4.865910,52.374555,2020-10-29 15:21:57
979,4.887960,52.359365,4.866404,52.371185,2020-10-29 15:05:27


(before you can query the server you need to run it)

sample query

[query synthax](http://dev.opentripplanner.org/apidoc/1.0.0/resource_PlannerResource.html)  
[response json structure](http://dev.opentripplanner.org/apidoc/1.0.0/json_Response.html)

In [40]:
row = df.sample(1).squeeze()
query = make_query(row)
r = requests.get(OTP_API, params=query)
route = parse_OTP_response(r.json())
route

{'success': True,
 'n_itineraries': 3,
 'duration': 2419,
 'walkDistance': 1356.2871278049915,
 'transfers': 0,
 'transitTime': 1302,
 'waitingTime': 2,
 'modes': [['WALK', 805, 992], ['TRAM', 1302, 4440], ['WALK', 310, 363]]}

#### visualization

In [41]:
plot(r.json()['plan'], color = 'green')

Trip from (4.8422,52.3452) to (4.8875,52.3740) at 1603981680000. 
3 connections found. 
Best one is 40min (1356m walk, 0 transfer(s), wait time 0.03min)

LEG 	 MODE 	DIST 	TIME
-----------------------------
1	WALK	992	805
2	TRAM	4441	1302
3	WALK	364	310


output

In [45]:
pd.read_csv('georequests_PT.csv', index_col=[0]).sample(5)

,success,n_itineraries,duration,walkDistance,transfers,transitTime,waitingTime,modes
id,,,,,,,,
619,True,3,1089,539.499970,0,623,2,"[['WALK', 51, 67], ['TRAM', 623, 2238], ['WALK..."
504,True,3,1496,1853.007188,0,0,0,"[['WALK', 1496, 1852]]"
211,True,3,1802,668.059135,0,1264,2,"[['WALK', 363, 453], ['BUS', 1264, 6004], ['WA..."
512,True,3,377,327.598910,0,83,2,"[['WALK', 45, 46], ['TRAM', 83, 314], ['WALK',..."
64,True,3,1141,1450.037489,0,0,0,"[['WALK', 1141, 1449]]"


---